# Sentiment Analyzer

In [1]:
import nltk
import numpy as np
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

/Users/MarkVII/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'absl'

In [2]:
wordnet_lemmatizer = WordNetLemmatizer()

def my_tokenizer(sentence):
    sentence = sentence.lower()
    tokens = nltk.tokenize.word_tokenize(sentence)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords.words('english') and t != '']
    return tokens

In [3]:
# Mac Os
# positive_reviews = BeautifulSoup(open('/Users/MarkVII/Dropbox/Projetos/Anaconda/data/sorted_data_acl/electronics/positive.review').read())
# positive_reviews = positive_reviews.findAll('review_text')

# negative_reviews = BeautifulSoup(open('/Users/MarkVII/Dropbox/Projetos/Anaconda/data/sorted_data_acl/electronics/negative.review').read())
# negative_reviews = negative_reviews.findAll('review_text')

# Windows
positive_reviews = BeautifulSoup(open('C://Users//sgtab//Dropbox//Projetos//Anaconda//data//sorted_data_acl//electronics//positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

negative_reviews = BeautifulSoup(open('C://Users//sgtab//Dropbox//Projetos//Anaconda//data//sorted_data_acl//electronics//negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

np.random.shuffle(positive_reviews)

c:\users\sgtab\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\sgtab\appdata\local\programs\python\python36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [4]:
positive_reviews = positive_reviews[:len(negative_reviews)]

In [5]:
word_index_map = {}
current_index = 0

positive_tokenized = []
negative_tokenized = []

In [6]:
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

print(word_index_map)     

{'previously': 0, 'owned': 1, '1gb': 2, 'lexar': 3, '40x': 4, 'card': 5, 'thought': 6, 'wa': 7, 'fastest': 8, 'could': 9, 'get': 10, 'completely': 11, 'wrong': 12, 'one': 13, 'fast': 14, 'sucker': 15, 'hold': 16, 'enormous': 17, 'amount': 18, 'image': 19, 'jpeg': 20, 'mode': 21, 'able': 22, 'store': 23, '1000': 24, 'raw': 25, 'converting': 26, 'thanks': 27, 'large': 28, 'capacity': 29, '444': 30, '20d': 31, 'thinking': 32, 'buying': 33, "n't": 34, 'hesitate': 35, 'awsome': 36, 'piece': 37, 'equipment': 38, 'heck': 39, 'investment': 40, 'far': 41, 'havent': 42, 'filled': 43, 'disc': 44, 'yet': 45, 'using': 46, 'came': 47, 'would': 48, 'fill': 49, 'minute': 50, 'talking': 51, 'great': 52, 'run': 53, 'shutter': 54, 'camera': 55, 'non': 56, 'stop': 57, 'full': 58, 'frame': 59, 'rate': 60, '3.5': 61, 'fps': 62, 'lag': 63, 'slow': 64, 'amazing': 65, 'make': 66, 'perform': 67, 'better': 68, 'canon': 69, 'rebel': 70, 'avoid': 71, 'burning': 72, 'coaster': 73, 'update': 74, 'firm': 75, 'ware': 

In [7]:
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum()
    x[-1] = label
    return x

In [8]:
N = len(positive_tokenized) + len(negative_tokenized)
data = np.zeros((N, len(word_index_map) + 1))
i = 0
                
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1
    
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1
    
print("Rows: ", len(data))
print("Columns: ", len(data[0]))

Rows:  2000
Columns:  11289


In [9]:
np.random.shuffle(data)

In [10]:
# verify data
X = data[:, :-1]
Y = data[:, -1:]

X_train = X[:-100,]
Y_train = Y[:-100,]

X_test = X[-100:,]
Y_test = Y[-100:,]

In [11]:
model = LogisticRegression()
model.fit(X_train, Y_train)
print("Classification rate: ", model.score(X_test, Y_test))

Classification rate:  0.7


c:\users\sgtab\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
